In [2]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [3]:
print("Data read started...")
data = pd.read_csv("result_nback.csv")
data = data.as_matrix()
print ("Data read finished.")
#Transform data

print(data.shape)

Data read started...
Data read finished.
(828, 11)


# Eliminate EEG data

In [4]:
data = np.delete(data, 1, 1) #timestamp
data = np.delete(data, 2, 1) # EEG_F3_Alpha
data = np.delete(data, 3, 1) # EEG_F3_Beta
data = np.delete(data, 4, 1) # EEG_F3_Theta

data.shape

(828, 7)

In [5]:
data[1]

array(['nback gyakorlas', 286880.0, 170520.0, 933.41, 4.9275, 307.67,
       343.79], dtype=object)

# Dictionary for the levels

In [6]:
level=["nback gyakorlas","0back","1back","2back","3back"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [7]:
data[:, 1] = preprocessing.scale(data[:, 1])
data[:, 2] = preprocessing.scale(data[:, 2])
data[:, 3] = preprocessing.scale(data[:, 3])
data[:, 4] = preprocessing.scale(data[:, 4])
data[:, 5] = preprocessing.scale(data[:, 5])
data[:, 6] = preprocessing.scale(data[:, 6])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [8]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [17]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]


# Shuffle data

In [18]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [19]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [20]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(579, 6) (579, 5) (124, 6) (124, 5) (125, 6) (125, 5)


# Build the net

In [54]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [55]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 579 samples, validate on 124 samples
Epoch 1/150
579/579 [==============================] - 0s - loss: 1.5075 - acc: 0.3506 - val_loss: 1.4087 - val_acc: 0.3548
Epoch 2/150
579/579 [==============================] - 0s - loss: 1.4099 - acc: 0.4145 - val_loss: 1.3227 - val_acc: 0.4274
Epoch 3/150
579/579 [==============================] - 0s - loss: 1.3506 - acc: 0.4214 - val_loss: 1.2816 - val_acc: 0.4919
Epoch 4/150
579/579 [==============================] - 0s - loss: 1.3291 - acc: 0.4525 - val_loss: 1.2545 - val_acc: 0.4839
Epoch 5/150
579/579 [==============================] - 0s - loss: 1.3031 - acc: 0.4680 - val_loss: 1.2365 - val_acc: 0.4597
Epoch 6/150
579/579 [==============================] - 0s - loss: 1.2589 - acc: 0.4905 - val_loss: 1.1967 - val_acc: 0.5161
Epoch 7/150
579/579 [==============================] - 0s - loss: 1.2470 - acc: 0.4974 - val_loss: 1.1793 - val_acc: 0.4597
Epoch 8/150
579/579 [==============================] - 0s - loss: 1.2004 - acc: 0.5147

In [56]:
model.evaluate(x_test, y_test, batch_size=batch_size)

100/125 [=======================>......] - ETA: 0s

[0.69764662384986875, 0.7520000100135803]